### Importing Libraries

In [3]:
import numpy as np
import pandas  as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

### Loading Data

In [4]:
data = pd.read_excel("../Data/data_chg_rapid_all.xlsx")
data.head()

,Chargingevent,CPID,Connector,StartDate,StartTime,EndDate,EndTime,StartDate_num,StartTime_num,EndDate_num,EndTime_num,duration,TotalkWh,Cost,Site,Group,Model,Model1,weekday,charger_id
0,4262365,50692,2,2018-03-05,23:55:00,2018-03-06,00:01:00,43164,23.916667,43165,0.016667,6.0,4.76,0.0,"Dundee Ice Arena, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,rapid,1,4
1,4262362,50339,2,2018-03-05,23:39:00,2018-03-06,00:40:00,43164,23.650000,43165,0.666667,61.0,8.31,0.0,"Public Works Dept, Clepington Rd. Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,rapid,1,2
2,4262354,50339,2,2018-03-05,23:21:00,2018-03-05,23:26:00,43164,23.350000,43164,23.433333,5.0,1.43,0.0,"Public Works Dept, Clepington Rd. Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,rapid,1,2
3,4262349,50692,2,2018-03-05,22:49:00,2018-03-05,23:27:00,43164,22.816667,43164,23.450000,38.0,14.48,0.0,"Dundee Ice Arena, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,rapid,1,4
4,4262341,50911,2,2018-03-05,22:32:00,2018-03-05,22:59:00,43164,22.533333,43164,22.983333,27.0,7.44,0.0,"Queen Street Car Park, Broughty Ferry, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,rapid,1,5


### Checking for null values

In [5]:
data.isna().sum()

Chargingevent    0
CPID             0
Connector        0
StartDate        0
StartTime        0
EndDate          0
EndTime          0
StartDate_num    0
StartTime_num    0
EndDate_num      0
EndTime_num      0
duration         0
TotalkWh         0
Cost             4
Site             0
Group            0
Model            0
Model1           0
weekday          0
charger_id       0
dtype: int64

Only `Cost` column has 'nan' values.

Dropping `Cost` column since it has either has `0` or `nan` values and is not so useful for us.

In [6]:
data['Model1'].unique()

array(['rapid'], dtype=object)

Dropping `Model1` column which only contains value `rapid`

In [7]:
data.drop(columns = ['Cost', 'Model1'], axis = 1, inplace = True)

### Sorting the data 

In [8]:
data = data.sort_values(by=['CPID','Connector', 'StartDate', 'StartTime', 'EndDate', 'EndTime'])
data.head()

,Chargingevent,CPID,Connector,StartDate,StartTime,EndDate,EndTime,StartDate_num,StartTime_num,EndDate_num,EndTime_num,duration,TotalkWh,Site,Group,Model,weekday,charger_id
1347,7120839,50338,1,2018-03-23,12:35:00,2018-03-23,13:03:00,43182,12.583333,43182,13.050000,28.0,2.951,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,5,1
1340,7121028,50338,1,2018-03-23,13:13:00,2018-03-23,13:20:00,43182,13.216667,43182,13.333333,7.0,0.765,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,5,1
5143,7221636,50338,1,2018-05-01,15:51:00,2018-05-01,16:18:00,43221,15.850000,43221,16.300000,27.0,8.053,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,2,1
63,4259962,50338,2,2018-03-05,07:28:00,2018-03-05,07:54:00,43164,7.466667,43164,7.900000,26.0,0.570,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,1,1
51,4260697,50338,2,2018-03-05,10:47:00,2018-03-05,11:19:00,43164,10.783333,43164,11.316667,32.0,10.010,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,1,1


### Checking for Anomalies

> We are checking if there are any anamolies in the start date and end date

> For instance, unexpected year or month, like `2018-01-19` and `2018-05-19` 

> So we change it to `2018-01-19` and `2018-01-19` 

In [9]:
def Date_Anomaly(data):
    wrong_date = []
    wrong_index = []
    for i in range(len(data) - 1):
        if data.iloc[i , 1] == data.iloc[i + 1, 1]:
            # start_time = datetime.strptime(data.iloc[i, 3], "%H:%M:%S")
            # end_time = datetime.strptime(data.iloc[i , 5], "%H:%M:%S")
            start_time = str(data.iloc[i , 3])
            end_time = str(data.iloc[i , 5])

            s_y = int(start_time.split('-')[0])    # Start Year
            s_m = int(start_time.split('-')[1])    # Start Month 
            
            e_y = int(end_time.split('-')[0])   # End Year
            e_m = int(end_time.split('-')[1])    # End Month 
        

            if (s_y == e_y) & (s_m != e_m):
                wrong_index.append(data.iloc[i].name)
                wrong_date.append(data.iloc[i])
                
    return wrong_date, wrong_index

In [10]:
wrong_date, wrong_index = Date_Anomaly(data)
len(wrong_date), len(wrong_index)

(5, 5)

> The dates we have received are the dates which are at their end of the month (basically moving to next month)
> So, we don't have to change them

### Adding `Occupancy Status` column into the dataset

In [11]:
data.insert(loc = 3, column = 'Occupancy Status', value = 1)

In [12]:
d1 = data.copy()

In [13]:
d1.columns

Index(['Chargingevent', 'CPID', 'Connector', 'Occupancy Status', 'StartDate',
       'StartTime', 'EndDate', 'EndTime', 'StartDate_num', 'StartTime_num',
       'EndDate_num', 'EndTime_num', 'duration', 'TotalkWh', 'Site', 'Group',
       'Model', 'weekday', 'charger_id'],
      dtype='object')

In [14]:
for i in range(len(data) - 1):
    if data.iloc[i , 1] == data.iloc[i + 1, 1]:
        new_row = [
                        data.iloc[i , 0],   # Chargingevent
                        data.iloc[i,  1],   # CPID
                        data.iloc[i , 2],   # Connector
                        0,                  # Occupancy Status
                        data.iloc[i , 6],   # StartDate
                        data.iloc[i , 7],   # StartTime 
                        data.iloc[i+1,4],   # EndDate      
                        data.iloc[i+1,5],   # EndTime
                        data.iloc[i , 10],   # StartDate_num
                        data.iloc[i , 11],   # StartTime_num
                        data.iloc[i+1, 8],   # EndDate_num 
                        data.iloc[i+1, 9],   # EndTime_num
                        data.iloc[i , 12],   # duration
                        data.iloc[i , 13],   # TotalkWh
                        data.iloc[i , 14],   # Site
                        data.iloc[i , 15],   # Group 
                        data.iloc[i , 16],   # Model
                        data.iloc[i , 17],   # weekday
                        data.iloc[i , 18]    #charger_id
                ]   
        d1.loc[len(d1) + i] = new_row

In [15]:
d2 = d1.copy() 

In [16]:
d2 = d2.sort_values(by = ['CPID', 'Connector', 'StartDate', 'StartTime', "EndDate"])
d2.head(10)

,Chargingevent,CPID,Connector,Occupancy Status,StartDate,StartTime,EndDate,EndTime,StartDate_num,StartTime_num,EndDate_num,EndTime_num,duration,TotalkWh,Site,Group,Model,weekday,charger_id
1347,7120839,50338,1,1,2018-03-23,12:35:00,2018-03-23,13:03:00,43182,12.583333,43182,13.050000,28.0,2.951,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,5,1
8870,7120839,50338,1,0,2018-03-23,13:03:00,2018-03-23,13:13:00,43182,13.050000,43182,13.216667,28.0,2.951,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,5,1
1340,7121028,50338,1,1,2018-03-23,13:13:00,2018-03-23,13:20:00,43182,13.216667,43182,13.333333,7.0,0.765,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,5,1
8872,7121028,50338,1,0,2018-03-23,13:20:00,2018-05-01,15:51:00,43182,13.333333,43221,15.850000,7.0,0.765,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,5,1
5143,7221636,50338,1,1,2018-05-01,15:51:00,2018-05-01,16:18:00,43221,15.850000,43221,16.300000,27.0,8.053,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,2,1
8874,7221636,50338,1,0,2018-05-01,16:18:00,2018-03-05,07:28:00,43221,16.300000,43164,7.466667,27.0,8.053,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,2,1
63,4259962,50338,2,1,2018-03-05,07:28:00,2018-03-05,07:54:00,43164,7.466667,43164,7.900000,26.0,0.570,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,1,1
8876,4259962,50338,2,0,2018-03-05,07:54:00,2018-03-05,10:47:00,43164,7.900000,43164,10.783333,26.0,0.570,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,1,1
51,4260697,50338,2,1,2018-03-05,10:47:00,2018-03-05,11:19:00,43164,10.783333,43164,11.316667,32.0,10.010,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,1,1
8878,4260697,50338,2,0,2018-03-05,11:19:00,2018-03-05,11:23:00,43164,11.316667,43164,11.383333,32.0,10.010,"Social Work Building, Jack Martin Way, Dundee",APT Controls; CYC; ChargePlace Scotland; Dunde...,APT Triple Rapid Charger,1,1


In [17]:
d2.to_csv("../Data/Data.csv")